In [ ]:
class Block(nn.Module):
    def __init__(self, input, output, p, kernel=3, padding=1, stride=1):
        super(Block, self).__init__()
        self.conv = nn.Conv2d(input, output, kernel, bias=False, padding=padding, stride=stride)
        self.BatchNorm2d = nn.BatchNorm2d(output)
        self.Dropout2d = nn.Dropout2d(p)
    def forward(self, x):
        out = self.conv(x)
        out = self.Dropout2d(out)
        out = self.BatchNorm2d(out)
        return out


class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()  # just run the init of parent class (nn.Module)
        self.block1 = Block(input=1, output=40, p=.5, kernel=5, padding=3)
        self.block2 = Block(input=40, output=30, p=.5, kernel=4, padding=2)
        self.block3 = Block(input=30, output=20, p=.5, kernel=3, padding=1)
        self.block4 = Block(input=20, output=10, p=.5, kernel=2, padding=0)

        x = torch.randn(28, 28).view(-1, 1, 28, 28)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 300, bias=False)  # flattening.
        self.BatchNorm1d_1 = nn.BatchNorm1d(300)
        self.Dropout1d = nn.Dropout(0.5)
        self.fc2 = nn.Linear(300, 10)

    def convs(self, x):
        out = F.max_pool2d(F.leaky_relu(self.block1(x), negative_slope=0.1), 2)
        x1 = out
        out = F.max_pool2d(F.leaky_relu(self.block2(out + x1), negative_slope=0.1), 2)
        x2 = out
        out = F.max_pool2d(F.leaky_relu(self.block3(out + x2), negative_slope=0.1), 2)
        out = F.max_pool2d(F.leaky_relu(self.block4(out), negative_slope=0.1), 2)
        if self._to_linear is None:
            self._to_linear = int(x[0].shape[0] * x[0].shape[1] * x[0].shape[2])
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.Dropout1d(x)
        x = self.BatchNorm1d_1(x)
        x = self.fc2(x)
        return x